<a href="https://colab.research.google.com/github/RepoRecipee/IIIT-PROJECTS-AND-LABS/blob/main/AIML_III_Module_3_Lab_3_Using_KNN_for_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Students Internship Batch of 2024
MODULE: CLASSIFICATION-1
# LAB-3 : Using KNN for Text Classification

Section 1: Understanding NLP tools

In this lab we will be using KNN on a real world NLP application i.e. is text classification. But first look at some NLP techniques for text classification and tools that we use when we want to use python for NLP.

# Section 1.2: Data Cleaning and Preprocessing step

Raw text must be processed and converted into a form so that it is suitable to use with various machine-learning algorithms.
In case of text, there are lots of things that need to be taken into account.

Removing numbers from the text
Handling capitalization and punctuation.
Stemming and Lemmatizing text.
And most importantly, one can't just use words or images directly in algorithms; they need to be converted into vectors- a form that algorithms can understand.

NLTK
NLTK (or Natural Language Tool Kit) is a commonly used library for processing text. We will use this tool in this lab. Lets first install it.

In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
import re
import numpy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bs4 import BeautifulSoup

def cleanText(text, lemmatize, stemmer):
    """Method for cleaning text from train and test data. Removes numbers, punctuation, and capitalization. Stems or lemmatizes text."""

    if isinstance(text, float):
        text = str(text)
    if isinstance(text, numpy.int64):
        text = str(text)
    try:
        text = text.decode()
    except AttributeError:
        pass

    soup = BeautifulSoup(text, "lxml")
    text = soup.get_text()
    text = re.sub(r"[^A-Za-z]", " ", text)
    text = text.lower()


    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()

        def get_tag(tag):
            if tag.startswith('J'):
                return wordnet.ADJ
            elif tag.startswith('V'):
                return wordnet.VERB
            elif tag.startswith('N'):
                return wordnet.NOUN
            elif tag.startswith('R'):
                return wordnet.ADV
            else:
                return ''

        text_result = []
        tokens = word_tokenize(text)  # Generate list of tokens
        tagged = pos_tag(tokens)
        for t in tagged:
            try:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0], get_tag(t[1][:2])))
            except:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0]))
        return text_result

    if stemmer:
        text_result = []
        tokens = word_tokenize(text)
        snowball_stemmer = SnowballStemmer('english')
        for t in tokens:
            text_result.append(snowball_stemmer.stem(t))
        return text_result

In [3]:
sample_text = "Troubling"
sample_text_result = cleanText(sample_text, lemmatize=False, stemmer=True)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text)
print(sample_text_result)
sample_text_result = cleanText(sample_text, lemmatize=True, stemmer=False)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text_result)

Troubling
troubl
trouble


# Section 1.2: BAG OF WORDS
A bag-of-words model, or BoW for short, is a way of extracting features from text for use in modeling, such as with machine learning algorithms.

The approach is very simple and flexible, and can be used in many ways for extracting features from documents.

A bag-of-words is a representation of text that describes the occurrence of words within a document. It is called a “bag” of words, because any information about the order or structure of words in the document is discarded. The model is only concerned with whether known words occur in the document, not where in the document.


[ ]


In [4]:
# Functions to convert document(s) to a list of words, with the option of removing stopwords. Returns document-term matrix.

def createBagOfWords(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = CountVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer = CountVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    bag_of_words_train = vectorizer.fit_transform(clean_train).toarray()
    bag_of_words_test = vectorizer.transform(clean_test).toarray()
    return bag_of_words_train, bag_of_words_test


Section 1.3: TF-IDF
TF-IDF technique is used to find meaning of sentences consisting of words and cancels out the incapabilities of Bag of Words technique which is good for text classification or for helping a machine read words in numbers.

The number of times a term occurs in a document is called its Term frequency (TF).

Document frequency is the number of documents in which the word is present. Inverse DF (IDF) is the inverse of the document frequency which measures the informativeness of term t.


[ ]


Section 2: UNDERSTANDING THE DATA : A REVIEWS DATASET
Sentiment analysis is the interpretation and classification of emotions (such as positive, negative and neutral) within text data using text analysis techniques.
Given below is a dataset consisting of reviews along with sentiment class (positive or negative).




In [5]:
# Upload the Reviews CSV file that has been shared with you.
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

Saving reviews.csv to reviews.csv


In [6]:
import pandas as pd
df = pd.read_csv('reviews.csv')

In [7]:
df = df.dropna()

In [8]:
df

,sentence,sentiment
0,Not sure who was more lost - the flat characte...,0
1,Attempting artiness with black & white and cle...,0
2,Very little music or anything to speak of.,0
3,The best scene in the movie was when Gerardo i...,1
4,"The rest of the movie lacks art, charm, meanin...",0
...,...,...
994,I just got bored watching Jessice Lange take h...,0
995,"Unfortunately, any virtue in this film's produ...",0
996,"In a word, it is embarrassing.",0
997,Exceptionally bad!,0


Section 3: KNN MODEL
Given below are two KNN models; in the first case we are using Bag-of-Words and in the second case we are using TF-IDF. Note the different metrics and parameters used in each.


[ ]


In [16]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd

def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    training_data = training_data.fillna('')  # Handle missing values
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)

    # Define different KNN configurations to test
    knn_params = [
        {"n_neighbors": 3, "weights": 'uniform', "metric": 'euclidean'},
        {"n_neighbors": 5, "weights": 'distance', "metric": 'manhattan'},
        {"n_neighbors": 7, "weights": 'uniform', "metric": 'minkowski', "p": 3},
        {"n_neighbors": 5, "weights": 'distance', "metric": 'chebyshev'}
    ]

    for params in knn_params:
        knn = neighbors.KNeighborsClassifier(
            n_neighbors=params.get("n_neighbors", 5),
            weights=params.get("weights", 'uniform'),
            algorithm='auto',
            leaf_size=30,
            p=params.get("p", 2),
            metric=params.get("metric", 'euclidean'),
            n_jobs=-1
        )

        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        acc = metrics.accuracy_score(y_test, predicted)
        print(f'KNN with BoW accuracy = {acc * 100:.2f}% using {params}')

        scores = cross_val_score(knn, X_train, y_train, cv=3)
        print(f"Cross Validation Accuracy: {scores.mean():.2f}")
        print(scores)
        print('\n')

    return predicted, y_test


TF-IDF KNN with Different Parameters and Distance Metrics

In [17]:
def tfidf_knn():
    """Method for determining nearest neighbors using TF-IDF and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    training_data = training_data.fillna('')  # Handle missing values
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"], test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)

    # Define different KNN configurations to test
    knn_params = [
        {"n_neighbors": 3, "weights": 'uniform', "metric": 'cosine'},
        {"n_neighbors": 5, "weights": 'distance', "metric": 'euclidean'},
        {"n_neighbors": 7, "weights": 'uniform', "metric": 'manhattan'},
        {"n_neighbors": 5, "weights": 'distance', "metric": 'minkowski', "p": 3}
    ]

    for params in knn_params:
        knn = neighbors.KNeighborsClassifier(
            n_neighbors=params.get("n_neighbors", 5),
            weights=params.get("weights", 'uniform'),
            algorithm='brute',  # Using brute since cosine distance doesn't work with ball_tree or kd_tree
            leaf_size=30,
            p=params.get("p", 2),
            metric=params.get("metric", 'cosine'),
            n_jobs=-1
        )

        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        acc = metrics.accuracy_score(y_test, predicted)
        print(f'KNN with TF-IDF accuracy = {acc * 100:.2f}% using {params}')

        scores = cross_val_score(knn, X_train, y_train, cv=3)
        print(f"Cross Validation Accuracy: {scores.mean():.2f}")
        print(scores)
        print('\n')

    return predicted, y_test


In [18]:
predicted, y_test = bow_knn()


<ipython-input-2-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with BoW accuracy = 60.00% using {'n_neighbors': 3, 'weights': 'uniform', 'metric': 'euclidean'}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Cross Validation Accuracy: 0.52
[0.38951311 0.59022556 0.57518797]


KNN with BoW accuracy = 64.50% using {'n_neighbors': 5, 'weights': 'distance', 'metric': 'manhattan'}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Cross Validation Accuracy: 0.60
[0.61423221 0.56766917 0.62781955]


KNN with BoW accuracy = 67.50% using {'n_neighbors': 7, 'weights': 'uniform', 'metric': 'minkowski', 'p': 3}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Cross Validation Accuracy: 0.58
[0.60674157 0.57894737 0.56390977]


KNN with BoW accuracy = 46.50% using {'n_neighbors': 5, 'weights': 'distance', 'metric': 'chebyshev'}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Cross Validation Accuracy: 0.47
[0.40074906 0.4924812  0.51503759]




# Section 4: SPAM TEXT DATASET
Now let's use what we've learnt to classify texts as spam or not spam.

In [20]:
# Upload the spam text data CSV file that has been shared with you. You can also download the file from https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam.csv


In [21]:
import pandas as pd
df = pd.read_csv('spam.csv')
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ã¼ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [22]:
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})

In [23]:
df.head(5)

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [24]:
len(df)

5572

### TASK - 2: Tweak the models below and see results with different parameters and distance metrics.

In [28]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd

def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    # Load and preprocess the dataset
    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})

    # Optionally reduce the dataset size for quicker testing
    training_data = training_data.sample(frac=0.1, random_state=5)  # Use 10% of the data

    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)

    # Define KNN model with optimized parameters
    knn = neighbors.KNeighborsClassifier(
        n_neighbors=3,            # Adjusted to a smaller number for quicker computation
        weights='uniform',         # Try 'distance' for another variant
        algorithm='auto',
        leaf_size=30,
        p=2,
        metric='euclidean',
        n_jobs=2                   # Use 2 cores to limit CPU usage
    )

    # Fit the model
    knn.fit(X_train, y_train)

    # Predict and evaluate the model
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print(f'KNN with BoW accuracy = {acc * 100:.2f}%')

    # Optionally, skip cross-validation to speed up
    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print(f"Cross Validation Accuracy: {scores.mean():.2f}")
    print(scores)
    print('\n')

    return predicted, y_test

# Run the function
predicted, y_test = bow_knn()


<ipython-input-2-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with BoW accuracy = 87.50%
Cross Validation Accuracy: 0.87
[0.8590604  0.87162162 0.86486486]




In [29]:
# This cell may take some time to run
predicted, y_test = bow_knn()

<ipython-input-2-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with BoW accuracy = 87.50%
Cross Validation Accuracy: 0.87
[0.8590604  0.87162162 0.86486486]




# Questions to Think About and Answer

Why does the TF-IDF approach generally result in a better accuracy than Bag-of-Words ?



In [31]:
def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    # Load and preprocess the dataset
    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})

    # Optionally reduce the dataset size for quicker testing
    training_data = training_data.sample(frac=0.1, random_state=5)  # Use 10% of the data

    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)

    # Define KNN model with optimized parameters
    knn = neighbors.KNeighborsClassifier(
        n_neighbors=3,            # Adjusted to a smaller number for quicker computation
        weights='uniform',         # Try 'distance' for another variant
        algorithm='auto',
        leaf_size=30,
        p=2,
        metric='euclidean',
        n_jobs=2                   # Use 2 cores to limit CPU usage
    )

    # Fit the model
    knn.fit(X_train, y_train)

    # Predict and evaluate the model
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print(f'KNN with BoW accuracy = {acc * 100:.2f}%')

    # Optionally, skip cross-validation to speed up
    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print(f"Cross Validation Accuracy: {scores.mean():.2f}")
    print(scores)
    print('\n')

    return predicted, y_test


Can you think of techniques that are better than both BoW and TF-IDF ?

In [32]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize input text
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors='pt')

# Get BERT embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Extract embeddings
last_hidden_states = outputs.last_hidden_state
print(last_hidden_states)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tensor([[[-0.0824,  0.0667, -0.2880,  ..., -0.3566,  0.1960,  0.5381],
         [ 0.0310, -0.1448,  0.0952,  ..., -0.1560,  1.0151,  0.0947],
         [-0.8935,  0.3240,  0.4184,  ..., -0.5498,  0.2853,  0.1149],
         ...,
         [-0.2812, -0.8531,  0.6912,  ..., -0.5051,  0.4716, -0.6854],
         [-0.4429, -0.7820, -0.8055,  ...,  0.1949,  0.1081,  0.0130],
         [ 0.5570, -0.1080, -0.2412,  ...,  0.2817, -0.3996, -0.1882]]])
